In [0]:
# !pip install tables --upgrade
# !pip install eli5
# !pip install xgboost
# !pip install hyperopt

In [0]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car/'

In [0]:
import re
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

#Wczytanie danych

In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [9]:
# Skupiam się jedynie na samochodach używanych
df = df[df['param_stan'] == 'Używane']
df.shape

(96920, 155)

#Feature Engineering

Druga iteracja

In [0]:
def extract_number(text):
  if text:
    return int(''.join(re.findall('[0-9]+', str(text))))
  else:
    return -1

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: extract_number(x))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: extract_number(x))

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: extract_number(x))

In [0]:
# zamiana euro na pln
df['price_value_pln'] = df.apply(lambda x: x.price_value if x.price_currency == 'EUR' else x.price_value, axis = 1)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat].iloc[0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [16]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

len(cat_feats)

151

In [17]:
feats = [x for x in cat_feats if x not in ['param_rok-produkcji__cat', 'param_moc__cat', 'param_pojemność-skokowa__cat']]
feats += ['param_rok-produkcji', 'param_moc', 'param_pojemność-skokowa']
len(feats)

151

#XGBoost

In [0]:
def run_model(data, model, feats):
  X = data[feats].values
  y = data['price_value_pln'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [24]:
feats_top = [
'param_rok-produkcji',
'param_moc',
'param_pojemność-skokowa',
'param_marka-pojazdu__cat',
'param_rodzaj-paliwa__cat',
'param_napęd__cat',
'param_uszkodzony__cat',
'param_typ__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'param_wersja__cat',
'param_faktura-vat__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_model-pojazdu__cat',
'param_skrzynia-biegów__cat',
'param_bezwypadkowy__cat',
'feature_kamera-cofania__cat',
'param_kraj-pochodzenia__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'seller_address__cat',
'feature_system-start-stop__cat',
'param_liczba-miejsc__cat',
'feature_regulowane-zawieszenie__cat',
'feature_nawigacja-gps__cat',
]

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0,
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
}

model = xgb.XGBRegressor(**xgb_params)
run_model(df, model, feats_top)

(-8085.877362856722, 97.17168138984546)

# Hyperopt

In [25]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(df, xgb.XGBRegressor(**params), feats_top)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space
xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5,16,1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
    'n_estimators': 100,
    'seed': 0,
}

# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75, 'tree_method': 'gpu_hist'}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65, 't

{'colsample_bytree': 0.9500000000000001,
 'learning_rate': 2,
 'max_depth': 5,
 'subsample': 0.9}

# Final

In [36]:
feats_top = [
'param_rok-produkcji',
'param_moc',
'param_pojemność-skokowa',
'param_marka-pojazdu__cat',
'param_rodzaj-paliwa__cat',
'param_napęd__cat',
'param_uszkodzony__cat',
'param_typ__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'param_wersja__cat',
'param_faktura-vat__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_model-pojazdu__cat',
'param_skrzynia-biegów__cat',
'param_bezwypadkowy__cat',
'feature_kamera-cofania__cat',
'param_kraj-pochodzenia__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'seller_address__cat',
'feature_system-start-stop__cat',
'param_liczba-miejsc__cat',
'feature_regulowane-zawieszenie__cat',
'feature_nawigacja-gps__cat',
]

xgb_params = {
    'learning_rate': 0.1,
    'max_depth': 10,
    'subsample': 0.95,
    'colsample_bytree': 0.75,
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
    'n_estimators': 100,
    'seed': 0,
}

model = xgb.XGBRegressor(**xgb_params)
run_model(df, model, feats_top)

(-6509.555551541865, 58.86167883366248)